In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

In [ ]:
class CTRDataset(Dataset):
    def __init__(self, folder_path, transform=None, ctr_min=0.0, ctr_max=0.27):
        self.folder_path = folder_path
        self.image_files = os.listdir(folder_path)
        self.transform = transform
        self.ctr_min = ctr_min
        self.ctr_max = ctr_max

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        filename = self.image_files[idx]
        ctr, _, _ = filename.split("_", 2)
        ctr = float(ctr) / 100.0
        ctr = (ctr - self.ctr_min) / (self.ctr_max - self.ctr_min)
        img_path = os.path.join(self.folder_path, filename)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor([ctr], dtype=torch.float32)

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = CTRDataset("filtered_images", transform)
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [4]:

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 1)
)

C:\Users\benas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\benas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)

for epoch in range(10):
    model.train()
    running_loss = 0
    for imgs, targets in train_loader:
        imgs, targets = imgs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0
    all_preds, all_targets = [], []
    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            all_preds.extend(outputs.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)

    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

Epoch 1, Train Loss: 0.0072, Val Loss: 0.0108, MAE: 0.0628, R²: 0.0062
Epoch 2, Train Loss: 0.0064, Val Loss: 0.0105, MAE: 0.0638, R²: 0.0101
Epoch 3, Train Loss: 0.0069, Val Loss: 0.0108, MAE: 0.0625, R²: -0.0159
Epoch 4, Train Loss: 0.0071, Val Loss: 0.0119, MAE: 0.0691, R²: -0.1067
Epoch 5, Train Loss: 0.0068, Val Loss: 0.0131, MAE: 0.0700, R²: -0.2014
Epoch 6, Train Loss: 0.0062, Val Loss: 0.0087, MAE: 0.0580, R²: 0.1647
Epoch 7, Train Loss: 0.0070, Val Loss: 0.0141, MAE: 0.0684, R²: -0.3453
Epoch 8, Train Loss: 0.0065, Val Loss: 0.0103, MAE: 0.0575, R²: 0.0126
Epoch 9, Train Loss: 0.0070, Val Loss: 0.0130, MAE: 0.0653, R²: -0.2236
Epoch 10, Train Loss: 0.0055, Val Loss: 0.0155, MAE: 0.0811, R²: -0.4279
